In [1]:
import numpy as np
import pandas as pd

In [2]:
df_dataset = pd.read_csv('dataset.csv')

In [3]:
df_dataset

,毕业学校是否985,学历,技能,是否被录取
0,Yes,本科,C++,No
1,Yes,本科,Java,Yes
2,No,硕士,Java,Yes
3,No,硕士,C++,No
4,Yes,本科,Java,Yes
5,No,硕士,C++,No
6,Yes,硕士,Java,Yes
7,Yes,博士,C++,Yes
8,No,博士,Java,Yes
9,No,本科,Java,No


In [4]:
dataset = df_dataset.values

In [5]:
dataset_985 = [1 if i == 'Yes' else 0 for i in dataset[:, 0]]
dataset_skill = [1 if i == 'C++' else 0 for i in dataset[:, 2]]
dataset_enroll = [1 if i == 'Yes' else 0 for i in dataset[:, 3]]
dataset_degree  = []
for data in dataset[:, 1]:
    if data == '本科':
        dataset_degree.append(1)
    elif data == '硕士':
        dataset_degree.append(2)
    else:
        dataset_degree.append(3)
df_dataset_wished = pd.DataFrame(data={
    '985': dataset_985,
    'degree': dataset_degree,
    'skill': dataset_skill,
    'enroll': dataset_enroll
})

In [6]:
df_dataset_wished

,985,degree,skill,enroll
0,1,1,1,0
1,1,1,0,1
2,0,2,0,1
3,0,2,1,0
4,1,1,0,1
5,0,2,1,0
6,1,2,0,1
7,1,3,1,1
8,0,3,0,1
9,0,1,0,0


In [7]:
dataset_wished = df_dataset_wished.values
x = dataset_wished[:, 0:3]
y = dataset_wished[:, 3]

In [8]:
class Bayes:
    
    def __init__(self):
        pass
    
    def fit(self, dataset, labels):
        labels_count, feature_count = len(labels), dataset.shape[1]
        datasets = np.hstack((dataset, labels.reshape((labels_count, 1))))
        p_labels, p_conditions = {}, {}
        # 类别数
        labels_type_count = len(set(labels))
        
        for label in set(labels):            
            dataset_label = datasets[datasets[:, -1] == label]
            dataset_label_length = dataset_label.shape[0]
            # 计算先验概率
            p_label = (dataset_label_length + 1) / (labels_count + labels_type_count)
            p_labels[label] = p_label
            p_conditions[label] = {}
            
            # 计算条件概率
            for feature in range(feature_count):
                p_conditions[label][feature] = {}
                dataset_label_feature = dataset_label[:, feature]
                # 当前特征的取值数
                feature_value_count = len(set(dataset_label_feature))
                for feature_value in set(dataset_label_feature):
                    p_conditions[label][feature][feature_value] = (dataset_label_feature[dataset_label_feature==feature_value].shape[0] + 1) / (dataset_label_length + feature_value_count)
        self.p_labels_ = p_labels
        self.p_conditions_ = p_conditions
    
    def predict(self, dataset):
        result = []
        
        for data in dataset:
            p = []
            for label in set(self.p_labels_):
                p_label = np.log(self.p_labels_[label])
                p_condition = 0
                feature_index = 0
                for feature_value in data:
                    p_condition += np.log(self.p_conditions_[label][feature_index][feature_value])
                    feature_index += 1
                p.append(p_label + p_condition)
            result.append(np.argmax(p))
        return result

In [9]:
bayes = Bayes()
bayes.fit(x, y)
print(bayes.predict([[1, 1, 1], [0, 1, 1]]))

[0, 0]


In [10]:
bayes.p_labels_

{0: 0.4166666666666667, 1: 0.5833333333333334}

In [11]:
bayes.p_conditions_

{0: {0: {0: 0.6666666666666666, 1: 0.3333333333333333},
  1: {1: 0.5, 2: 0.5},
  2: {0: 0.3333333333333333, 1: 0.6666666666666666}},
 1: {0: {0: 0.375, 1: 0.625},
  1: {1: 0.3333333333333333, 2: 0.3333333333333333, 3: 0.3333333333333333},
  2: {0: 0.75, 1: 0.25}}}